
___________
### 0. Delete Database And Tables

In [1]:
import os
import sqlite3

# Close any existing connections
conn = sqlite3.connect('nursery.db')
conn.close()

# Delete the file completely
if os.path.exists('nursery.db'):
    os.remove('nursery.db')
    print("Database file removed completely!")
else:
    print ("Database file did not exist.")

Database file removed completely!


In [ ]:
import sqlite3

# Connect to your database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

try:
    # Corrected execute() spelling and added IF EXISTS clause
    cursor.execute('''
        DROP TABLE IF EXISTS NURSERY_PESTC
    ''')
    conn.commit()  # Important: Save the changes
    print("Table dropped successfully")
    
except sqlite3.Error as e:
    print(f"Error dropping table: {e}")
    
finally:
    conn.close()  # Always close the connection

______________________
### 1. Create Database File And Table Structure

In [ ]:
import sqlite3

# Create or connect to the sql database file
conn = sqlite3.connect('nursery.db')
cursor=conn.cursor() # Once you open the connection
conn.close()  # Always close the connection


In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect('nursery.db')
cursor = conn.cursor()

try:
    # Create the species table structure
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SPECIES (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            SCIENTIFIC_NAME TEXT NOT NULL UNIQUE,
            COMMON_NAME TEXT,
            FAMILY TEXT,
            GROWTH_RATE TEXT CHECK(GROWTH_RATE IN ('Slow', 'Moderate', 'Fast')),
            WATER_NEEDS TEXT CHECK(WATER_NEEDS IN ('Low', 'Medium', 'High')),
            LIGHT_REQUIREMENTS TEXT CHECK(LIGHT_REQUIREMENTS IN ('Full Sun', 'Partial Shade', 'Shade')),
            NOTES TEXT
        )            
    ''')

    # Create the Batches Table with foreign key constraint
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS BATCH (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            SPECIES_ID INTEGER NOT NULL,
            BATCH_CODE TEXT NOT NULL UNIQUE,
            START_DATE DATE NOT NULL,
            INITIAL_COUNT INTEGER NOT NULL,
            CURRENT_COUNT INTEGER,
            STATUS TEXT CHECK(STATUS IN ('Active', 'Archived', 'Planted')),
            NOTES TEXT,
            FOREIGN KEY (SPECIES_ID) REFERENCES SPECIES(ID)
        )
    ''')


    conn.commit()
    print("Tables created successfully")

except sqlite3.Error as e:
    print(f"Error creating tables: {e}")
    conn.rollback()

finally:
    conn.close()

Tables created successfully


In [ ]:
# Create the preliminar table structure
cursor.execute('''
    CREATE TABLE NURSERY_PESTC (
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                BATCH_CODE TEXT NOT NULL,               -- Código único del lote (ej: FRT-2024-08)
                TREE_SPECIES TEXT NOT NULL,             -- Nombre científico (ej: Fraxinus excelsior)
                PEST_TYPE TEXT NOT NULL,                -- Tipo de plaga (ej: Tetranychus urticae)
                INFESTATION_LEVEL TEXT CHECK(INFESTATION_LEVEL IN ('Leve', 'Moderado', 'Grave')), 
                TREE_COUNT_TOTAL INTEGER NOT NULL,
                TREE_COUNT_AFFECTED INTEGER NOT NULL,
                BIOLOGICAL_CONTROL_USED TEXT,           -- Agente biológico (ej: Amblyseius swirskii)
                CHEMICAL_PRODUCT_NAME TEXT,             -- Nombre comercial (ej: Acetamiprid 20%)
                APPLICATION_METHOD TEXT,                -- (ej: Pulverización foliar, Riego al suelo)
                PRODUCT_DOSE REAL,                      -- Dosis exacta (ej: 2.5 ml/L)
                APPLICATION_DATE DATE NOT NULL

    )            
''')